In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score


In [3]:
df = pd.read_csv('input_for_modeling.csv')
print(df.shape)
df.head(3)

(864, 73)


,title,company,benefit_rating,overall_rating,culture_rating,description,jobsecurity_rating,mgmt_rating,worklife_rating,city,...,Web Scraping,Decision Tree,Random Forest,Clustering,corp,PHD,Masters,Bachelors,role_relevance,salary
0,Data Scientist,Hillsborough County,3.6,3.8,3.7,Hillsborough County is looking for a Data Scie...,3.6,3.5,3.8,Tampa,...,False,False,False,False,hillsborough county looking data tampa fl clic...,False,False,False,True,88161.0
1,Data Scientist,"Hillsborough County, FL",3.6,3.8,3.7,Desired: Machine LearningRSQLT-SQLPL/SQLMATLAB...,3.6,3.5,3.8,Tampa,...,False,False,False,False,desired machine learningrsqltsqlplsqlmatlabdat...,False,True,True,True,88161.0
2,Data Scientist,BICP,5.0,5.0,5.0,"BICP, a market leader in Advanced Analytics, D...",4.5,5.0,5.0,Tampa,...,False,True,False,True,bicp market leader advanced analytics data sci...,True,False,True,True,145000.0


In [5]:
#No nulls in salary field
df['salary'].isnull().sum()

0

In [7]:
#Number of uniques in columns :
for c in ['title','company', 'city']:
    print(c, df[c].nunique())

title 605
company 405
city 41


In [9]:
df['salary_range'] = [1 if a > df['salary'].median() else 0 \
                     for a in df['salary']]

In [22]:
x_desc = df.description
y = df.salary_range

In [24]:
cv = CountVectorizer(stop_words="english")
cv.fit(x_desc)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [25]:
len(cv.get_feature_names())

12980

In [49]:
x_desc_mat = pd.DataFrame(cv.transform(x_desc).todense(), columns=cv.get_feature_names())
x_train, x_test, y_train, y_test = train_test_split(np.asmatrix(x_desc_mat), y, test_size=0.3,
                                                  random_state=1, stratify=y)
word_counts = x_desc_mat.sum(axis=0)
word_counts.sort_values(ascending = False).head(20)

data            5630
experience      4494
research        2409
work            2244
required        1842
science         1582
position        1558
skills          1503
team            1496
years           1471
development     1282
information     1265
degree          1263
education       1153
requirements    1144
time            1139
business        1096
management      1093
new             1021
analysis        1010
dtype: int64

In [50]:
model_rf = RandomForestClassifier(200, random_state=59)
model_rf.fit(x_train, y_train)

model_pred = model_rf.predict(x_test)
acc = accuracy_score(y_test, model_pred)
print("Accuracy Score:", acc.round(3))

cv_score = cross_val_score(model_rf, x_desc_mat.as_matrix(), y.as_matrix(), cv=5, n_jobs=-1)
print("Cross Validation Score: {:0.3} ± {:0.3}".format(cv_score.mean(), cv_score.std()))

Accuracy Score: 0.746
Cross Validation Score: 0.74 ± 0.0588
